In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
import json
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']
Using AI model: openai:gpt-4o


In [3]:
class Task(BaseModel):
    type: str = Field(..., description=(
        'The type of task. '
        'For example: "formal", "conversational", "hybrid", ...'))
    description: str = Field(
        ...,
        description='Clear description for executing this task.'
    )


class OrchestratorResponse(BaseModel):
    analysis: str = Field(..., description=(
        'Explain your understanding of the task and which variations '
        'would be valuable. Focus on how each approach serves '
        'different aspects of the task.'
    ))
    tasks: List[Task] = Field(..., description="List of tasks")


async def orchestrate(task: str) -> Dict:
    """Process task by breaking it down and running subtasks in parallel."""

    orchestrator_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'Analyze this task and break it down into ',
            '2-3 distinct approaches.',
        ),
        result_type=OrchestratorResponse,
    )
    orchestrator_response = await orchestrator_agent.run(task)
    
    analysis = orchestrator_response.data.analysis
    tasks = orchestrator_response.data.tasks
    
    show('', title='Orchestrator Output')
    show(analysis, title='Analysis')
    show([task.model_dump() for task in tasks], title='Tasks')
    
    # Process all the tasks in parallel and collect results
    worker_agent = Agent(
        AI_MODEL,
        system_prompt='Generate content based on the task specification.',
    )
    worker_responses = await asyncio.gather(*[
        worker_agent.run(json.dumps(
            {'original_task': task} | task_info.model_dump()
        ))
        for task_info in tasks
    ])
    worker_results = [
        {
            'type': task.type,
            'description': task.description,
            'result': response.data,
        }
        for task, response in zip(tasks, worker_responses)
    ]

    for worker_result in worker_results:
        show(
            worker_result['result'],
            title=f"Worker Result ({worker_result['type']})",
        )
    
    return {
        "analysis": analysis,
        "worker_results": worker_results,
    }

In [4]:
results = await orchestrate(
    'Write a product description for a new eco-friendly water bottle.'
)

show(results, title='Final Results')


Orchestrator Output
-------------------


Analysis
--------

The task of writing a product description for an eco-friendly water bottle can be approached in various ways to cater to different audiences or marketing strategies. Each approach serves a distinct purpose and engages different aspects of the task: 1) A formal approach might appeal to environmentally conscious consumers and emphasize the technical specifications and certifications of the product. 2) A conversational approach could create a relatable and engaging tone, focusing on everyday use and benefits, appealing to a broader audience. 3) A hybrid approach combines the in-depth information found in formal descriptions with the engaging tone of conversational descriptions to appeal to a wide audience, providing both detailed information and a personalized touch.


Tasks
-----

[{'description': 'Craft a product description that highlights the eco-friendly '
                 'aspects of the water bottle, such as materials us